In [3]:
import tensorflow as tf
import os

In [28]:
with open('fake_sentences.txt') as f:
    lines = f.readlines()
    
index=0
for line in lines:
    file_name = str(index)+'fake'
    index+=1
    with open(file_name, 'a') as the_file:
        the_file.write(line+'\n')
    
with open('sentence_good_format.txt') as f:
    lines = f.readlines()

index=0
for line in lines:
    file_name = str(index)+'real'
    index+=1
    
    with open(file_name, 'a') as the_file:
        the_file.write(line+'\n')

In [40]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import numpy as np
from collections import Counter
import os

"""
    first :
            i will make a Dictionary So it contains every Word on the Document 
            and Then Remove unwanted items .
            unwanted items => Less Common and Have Numbers .
            
            So Lets Do a Function Called dic_process()
"""

TRAIN_DIR = "./train"
TEST_DIR = "./test"

def dic_process(root_dir):
    all_words = []
    emails= [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
    for email in emails :
        with open (email) as m:
            for line in m :
                words = line.split()
                all_words+=words
    dictionary = Counter(all_words)
    list_to_remove = list(dictionary.keys())

    for item in list_to_remove:

        if item.isalpha() == False:
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    dictionary = dictionary.most_common()

    return dictionary

In [75]:
"""
    Second :
            i will extract the features and put them in a matrix
            
            So Lets Do a Function Called ext_features()
"""

def ext_features(mail_dir):
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    feature_matrix = np.zeros(len(files))
    train_labels = np.zeros(len(files))
    count = 0
    docId = 0

    for fil in files :
        with open(fil) as fi:
            for i,line in enumerate(fi):
                if i == 0 :
                    words = line.split()
                    for word in words :
                        wordId=0
                        for i,d in enumerate(dictionary):
                            if d[0] == word:
                                wordId = i
                                feature_matrix[docId] = words.count(word)
            train_labels[docId] = 0
            filepathTokens = fil.split('/')
            
            lastToken = filepathTokens[len(filepathTokens) - 1]
            
            if 'real' in lastToken:
                #print docId
                train_labels[docId] = 1
                count = count + 1
            docId = docId+1
    print feature_matrix,train_labels
    return feature_matrix , train_labels

In [76]:
dictionary = dic_process(TRAIN_DIR)
print ("reading and processing emails from file.")
features_matrix =np.reshape(ext_features(TRAIN_DIR),(406,1))
labels =np.reshape(ext_features(TRAIN_DIR),(406,1))

test_feature_matrix =np.reshape(ext_features(TEST_DIR),(54,1))
test_labels = np.reshape(ext_features(TEST_DIR),(54,1))

clf = GaussianNB()
print("Training Model :")
clf.fit(features_matrix,labels)
predicted_labels = clf.predict(test_feature_matrix)

print (accuracy_score(test_labels,predicted_labels))


reading and processing emails from file.
[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 2. 3. 2. 1. 0. 1. 3.
 2. 2. 1. 1. 2. 1. 2. 4. 2. 2. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 4. 1. 5.
 1. 1. 1. 1. 1. 1. 1. 4. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 5. 1. 1. 2. 1. 1. 1. 2. 1. 7. 1. 1. 1. 1. 1. 4. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 0. 1. 1. 1. 4. 1. 1. 1.
 1. 1. 1. 1. 3. 1. 3. 8. 1. 2. 1. 1. 1. 1. 1. 1. 1. 3. 2. 2. 1. 1. 0. 1.
 1. 2. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 3.
 1. 3. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 3. 1. 8. 2. 1. 1. 2. 1. 1. 1. 1.
 1. 3. 1. 1. 1. 2. 1. 1. 1. 1. 0.] [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.